In [74]:
import pandas as pd
import numpy as np
#from pandas import ExcelWriter
#from pandas import ExcelFile
#from scipy import stats
import matplotlib.pyplot as plt
import math
import xlwt
import openpyxl

In [75]:
filename = 'GCData_TE_ML.xlsx'

In [76]:
# FAME Retention Times (minutes on Restek Column with Restek protocol)

C7 = 2.75
C8 = 3.9
C9 = 5.4
C10_1 = 6.93
C10 = 7.0
C11 = 8.5
C12_1 = 9.55
C12 = 9.85 
C13 = 11.15 # did not shift, but other peaks did
C12OH = 11.75

C14_1 = 12.05
C14 = 12.25 
C15 = 13.4
C14OH = 13.8
C16_1 = 14.2
C16 = 14.4
C17_cp = 15.3
C17 = 15.4
C18_1 = 16.13
C18 = 16.35


# Create a dictionary which can be updated by changing values above. The dictionary will be called in peak identification (next block)

RT_dict = {'C7': C7, 'C8': C8, 'C9': C9, 'C10_1': C10_1, 'C10': C10, 'C11': C11, 'C12_1': C12_1, 'C12': C12, 'C13': C13, 'C14_1': C14_1, 'C14': C14, 'C15': C15, 'C16_1': C16_1, 'C16': C16, 'C17': C17, 'C18_1':C18_1, 'C18':C18}

# Create list of analytes to make it easy to call dictionary in future
analytes = ['C7', 'C8', 'C9', 'C10_1', 'C10', 'C11', 'C12_1', 'C12', 'C13',#'C12OH', #
            'C14_1','C14', 'C15', 'C16_1', 'C16', #'C17_cp',
            'C17', 'C18_1', 'C18']
##########################
C7 = 2.63
C8 = 3.79
C9 = 5.29
C10 = 6.90
C11 = 8.39
C12 = 9.76
C13 = 11.02
C14 = 12.18
C15 = 13.27
C16 = 14.31
C17 = 15.29
C18 = 16.00

# Create a dictionary which can be updated by changing values above. The dictionary will be called in peak identification (next block)

RT_dict = {'C7': C7, 'C8': C8, 'C9': C9, 'C10': C10, 'C11': C11, 'C12': C12, 'C13': C13, 'C14': C14, 'C15': C15, 'C16': C16, 'C17': C17,'C18': C18}

# Create list of analytes to make it easy to call dictionary in future
analytes = ['C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18']

In [77]:
wb = openpyxl.load_workbook(filename)
sheet1 = ''
sheet1 = wb['John_Code']
sheet2 = ''
sheet2 = wb.create_sheet('Peak_ID')

In [78]:
df = pd.read_excel(filename, sheet_name = 'John_Code', header = None)


#Preparing list of strings called Vial_names where each element is the name of a sample

Vial_names = []
Standard_names = ['FAME1000', 'FAME500', 'FAME100', 'FAME50', 'FAME25']
Sample_names = ['ClFatB3','ClFatB3_tr','ClFatB3_tr_N28S','ClFatB3_tr_I65M']
Sample_names = ['133_222_227','BTE','D10S','E174G','H132Q','H204A','I198E','K159E','N131I','R109C','Rec']

for x in Sample_names:
    vial1 = x + '-1'
    Vial_names.append(vial1)
    vial2 = x + '-2'
    Vial_names.append(vial2)
    vial3 = x + '-3'
    Vial_names.append(vial3)
    
for x in Standard_names:
#    vial1 = x + '-lo'
    vial1=x
    Vial_names.append(vial1)
#    vial2 = x + '-hi'
#    vial2=x
#    Vial_names.append(vial2)

    

In [79]:
Vial_names

['133_222_227-1',
 '133_222_227-2',
 '133_222_227-3',
 'BTE-1',
 'BTE-2',
 'BTE-3',
 'D10S-1',
 'D10S-2',
 'D10S-3',
 'E174G-1',
 'E174G-2',
 'E174G-3',
 'H132Q-1',
 'H132Q-2',
 'H132Q-3',
 'H204A-1',
 'H204A-2',
 'H204A-3',
 'I198E-1',
 'I198E-2',
 'I198E-3',
 'K159E-1',
 'K159E-2',
 'K159E-3',
 'N131I-1',
 'N131I-2',
 'N131I-3',
 'R109C-1',
 'R109C-2',
 'R109C-3',
 'Rec-1',
 'Rec-2',
 'Rec-3',
 'FAME1000',
 'FAME500',
 'FAME100',
 'FAME50',
 'FAME25']

In [80]:
df

,0,1,2,3,4,5
0,133_222_227-1,NaN,NaN,NaN,NaN,NaN
1,Peak#,R.Time,I.Time,F.Time,Area,Height
2,1,2.118,2.082,2.203,1465,597
3,2,2.546,2.483,2.758,263509,117671
4,3,3.683,3.631,3.78,1754,717
...,...,...,...,...,...,...
1367,13,13.184,13.119,13.347,100615,47646
1368,14,14.219,14.15,14.416,120233,58214
1369,15,14.562,14.509,14.654,2815,1302
1370,16,15.2,15.144,15.41,99833,47608


In [81]:
# This block goes through the dataframe from the John_Code sheet and picks out sample names and relevant peaks.

num_rows = len(df)
row_name = list(range(0, num_rows))
txt_file_headers = ['Peak#', 'R.time', 'I.time', 'F.Time', 'Area', 'Peak_ID']

for i in row_name:                
    if i < num_rows+1: 
        # picks out each row in dataframe and truncates - would be more elegant to not have to use dataframe but whatever
        row_calling = df.loc[i,:]
        vals = row_calling.values.tolist()
        vals = vals[0:6]
        
        y = type(vals[0])
        
        if vals[0] in Vial_names:
            sample_of_interest = 'yes'
        
            # picks out sample names and adds to worksheet
            print(vals[0])
            heading = [vals[0],vals[0],vals[0],vals[0],vals[0],vals[0]]
            print(heading)
            sheet2.append(heading)
            sheet2.append(txt_file_headers)

        elif vals[0] != 'Peak#' and y == str:
            sample_of_interest = 'no'
            
                
        # picks out peaks
        x = type(vals[1])
        if x == int: # This if statement is for the rare occasion when the retention time of an analyte is an integer
            vals[1] = float(vals[1])
            x = type(vals[1])
            
        if x == float and sample_of_interest == 'yes':
            for a in analytes:
                upper_tolerance = RT_dict[a] + 0.2
                lower_tolerance = RT_dict[a] - 0.2
                if vals[1] < upper_tolerance and vals[1] > lower_tolerance:
                    #print(vals[1])
                    vals.append(a)
                    #print(vals)
                    sheet2.append(vals[1:])
                    

133_222_227-1
['133_222_227-1', '133_222_227-1', '133_222_227-1', '133_222_227-1', '133_222_227-1', '133_222_227-1']
133_222_227-2
['133_222_227-2', '133_222_227-2', '133_222_227-2', '133_222_227-2', '133_222_227-2', '133_222_227-2']
133_222_227-3
['133_222_227-3', '133_222_227-3', '133_222_227-3', '133_222_227-3', '133_222_227-3', '133_222_227-3']
BTE-1
['BTE-1', 'BTE-1', 'BTE-1', 'BTE-1', 'BTE-1', 'BTE-1']
BTE-2
['BTE-2', 'BTE-2', 'BTE-2', 'BTE-2', 'BTE-2', 'BTE-2']
BTE-3
['BTE-3', 'BTE-3', 'BTE-3', 'BTE-3', 'BTE-3', 'BTE-3']
D10S-1
['D10S-1', 'D10S-1', 'D10S-1', 'D10S-1', 'D10S-1', 'D10S-1']
D10S-2
['D10S-2', 'D10S-2', 'D10S-2', 'D10S-2', 'D10S-2', 'D10S-2']
D10S-3
['D10S-3', 'D10S-3', 'D10S-3', 'D10S-3', 'D10S-3', 'D10S-3']
E174G-1
['E174G-1', 'E174G-1', 'E174G-1', 'E174G-1', 'E174G-1', 'E174G-1']
E174G-2
['E174G-2', 'E174G-2', 'E174G-2', 'E174G-2', 'E174G-2', 'E174G-2']
E174G-3
['E174G-3', 'E174G-3', 'E174G-3', 'E174G-3', 'E174G-3', 'E174G-3']
FAME1000
['FAME1000', 'FAME1000', 'FA

In [82]:
wb.save(filename)

In [83]:
#Now remove duplicate peaks from Peak_ID tab

In [84]:
wb = openpyxl.load_workbook(filename)
sheet2 = ''
sheet2 = wb['Peak_ID']
sheet3 = ''
sheet3 = wb.create_sheet('Quantification w IS,ES')


In [85]:
ID  = 'blank'
Area = 'blank'
sheet3_list = []

for row in sheet2.iter_rows(min_row = 1, max_row = 5000, min_col = 1, max_col = 10, values_only = True): # iterates through a semi-arbitrary number of rows and only reports values, still a tuple
    
    if row[5] == ID: # If there is a duplicate label for a single peak, pick the peak with the larger area
        if row[3] > Area:
            sheet3_list.pop()
            sheet3_list.append(row)
        else:
            continue
            
    else:
        sheet3_list.append(row)
    
    if row[3] == None: # breaks out of for loop once all text file data is iterated through
        break
    
    else:
        ID = row[5]
        Area = row[3]
            
for row in sheet3_list:
    sheet3.append(row)
    
#Notice that this code may not work as intended if there is a duplicate in the first row, however, since the first row will always be a sample name, it is fine.

In [86]:
wb.save(filename)

In [87]:
wb = openpyxl.load_workbook(filename)
sheet_read = ''
sheet_read = wb['Peak_ID']
sheet3 = ''
sheet3 = wb.create_sheet('Quantification w IS,ES-full')

In [88]:
sample_header=[]
data_header = ['Peak#', 'R.time', 'I.time', 'F.time', 'Area', 'Peak_ID']

for row in sheet_read.iter_rows(min_row = 1, max_row = sheet_read.max_row, min_col = 1, max_col = 10, values_only = True): # iterates through a semi-arbitrary number of rows and only reports values, still a tuple
    if row[0] == row[1] and row [1] == row[2]: # This is a bit clunky, but it ensures the order of the samples in the Peak_ID tab is maintained
        sample = row[0]
        sample_header = [sample, sample, sample, sample, sample, sample]
        sheet3.append(sample_header)
        sheet3.append(data_header)
        for a in analytes:
            blank_row = [0,0,0,0,0,a]
            sheet3.append(blank_row)
            
        

In [89]:
print(sheet3.max_row)

532


In [90]:
sheet2 = ''
sheet2 = wb['Quantification w IS,ES']
sheet3 = ''
sheet3 = wb['Quantification w IS,ES-full']

In [91]:
Ass_Fat = True
last_count = 1
while Ass_Fat:
    
    counter = 0
    
    for row2, row3 in zip(sheet2.iter_rows(min_row = 1, max_row = sheet3.max_row, min_col = 1, max_col = 10, values_only = True), sheet3.iter_rows(min_row = 1, max_row = sheet3.max_row, min_col = 1, max_col = 10, values_only = True)):
        #print(row1[0], row2[0])
        counter = counter + 1
        #print(counter)
        if counter < last_count:
            continue
        elif row2[5] == row3[5]:
            last_count = counter
            print(last_count)
        elif row2[5] != row3[5]:
            #print(row3[5])
            sheet2.insert_rows(counter)
            sheet2.cell(row = counter, column = 6).value = row3[5]
            sheet2.cell(row = counter, column = 5).value = 0.000000001
            #last_count = counter
            break

        if counter == sheet3.max_row:
            Ass_Fat = False
            break
        else:
            continue

        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
205
206
206
207
207
208
209
209
210
210
211
211
212
212
213
213
214
214
215
215
216
216
217
217
218
218
219
220
221
222
223
224
224
225
225
226
226
227
227
228
228
229
229
230
230
231
231
232
232
233
233
234
234
235
235
236
236
237
237
238
238
239
239
240
240
241
241
242
242
243
243
244


In [92]:
#wb.remove_sheet('Quantification w IS,ES-full')
wb.save(filename)

In [1]:
for i, j in zip([1,2,3], [3,2,1]):
    print(i, j)

1 3
2 2
3 1
